In [10]:
import random
from reconchess import *
import json
import pprint
import torch
import re

In [5]:
game_history = GameHistory.from_file("./RandomBot-TroutBot-black-2025_03_18-06_50_14.json")

In [51]:
# Testing GameHistory methods

# Player names
print(f'White player name: {game_history.get_white_player_name()}')
print(f'Black Player name: {game_history.get_black_player_name()}')

first_move = list(game_history.turns())[0]
arbitrary_move = list(game_history.turns())[2]
next_move = list(game_history.turns())[3]
last_move = list(game_history.turns())[-1]

# Turns data
print(f'---------------------------------------------------------------------------------------------')
print(f'There were no turns this game: {game_history.is_empty()}')
print(f'There were {game_history.num_turns()} this game')
print(f'The list of turns this game are: {list(game_history.turns())}')
print(f'Is this the first turn? {game_history.is_first_turn(first_move)}')
print(f'First turn: {game_history.first_turn()}')
print(f'Is this the last turn? {game_history.is_last_turn(last_move)}')
print(f'Last turn: {game_history.last_turn()}')

# Winner utility
print(f'---------------------------------------------------------------------------------------------')
print(f'The winning color: {game_history.get_winner_color()}')
print(f'Win condition: {game_history.get_win_reason()}')

# Sensing Utility
print(f'---------------------------------------------------------------------------------------------')
print(f'Turn has sense: {game_history.has_sense(arbitrary_move)}')
print(f'Sense of a particular turn: {game_history.sense(arbitrary_move)}')
print(f'Sense result of a sensing action: {game_history.sense_result(arbitrary_move)}')

# Moving Utility
print(f'---------------------------------------------------------------------------------------------')
print(f'Turn has move: {game_history.has_move(arbitrary_move)}')
print(f'Requested move of turn: {game_history.requested_move(arbitrary_move)}')
print(f'Taken move of turn: {game_history.taken_move(arbitrary_move)}')
print(f'Square I captured an opponent piece: {game_history.capture_square(arbitrary_move)} [NOTE: May be none of no piece captured this turn]')
print(f'Summarize move phase of turn: {game_history.move_result(arbitrary_move)}')

# Ground Truth methods
print(f'---------------------------------------------------------------------------------------------')
print(f'Truth before move (Forsyth–Edwards Notation): {game_history.truth_fen_before_move(arbitrary_move)}')
print(f'Truth before move: {game_history.truth_board_before_move(arbitrary_move)}')
print(f'Truth after move: (Forsyth-Edwards Notatin): {game_history.truth_fen_after_move(arbitrary_move)}')
print(f'Truth after move: {game_history.truth_board_after_move(arbitrary_move)}')
print(f'Map function: {game_history.collect(game_history.sense, list(game_history.turns()))}')



White player name: RandomBot
Black Player name: TroutBot
---------------------------------------------------------------------------------------------
There were no turns this game: False
There were 24 this game
The list of turns this game are: [Turn(white, 0), Turn(black, 0), Turn(white, 1), Turn(black, 1), Turn(white, 2), Turn(black, 2), Turn(white, 3), Turn(black, 3), Turn(white, 4), Turn(black, 4), Turn(white, 5), Turn(black, 5), Turn(white, 6), Turn(black, 6), Turn(white, 7), Turn(black, 7), Turn(white, 8), Turn(black, 8), Turn(white, 9), Turn(black, 9), Turn(white, 10), Turn(black, 10), Turn(white, 11), Turn(black, 11)]
Is this the first turn? True
First turn: Turn(white, 0)
Is this the last turn? True
Last turn: Turn(black, 11)
---------------------------------------------------------------------------------------------
The winning color: False
Win condition: WinReason.KING_CAPTURE
---------------------------------------------------------------------------------------------
Turn

In [57]:
def replace_numbers_with_u(strings):
    def replacer(match):
        return 'u' * int(match.group())  # Convert number to integer and repeat 'u' that many times

    return [re.sub(r'\d+', replacer, s) for s in strings]

# Write function to convert FEN string to Tensor
def FEN2InputTensor(fen_string):
    my_color = 'white'
    fen_pattern = re.compile(r"^([rnbqkpRNBQKP1-8/]+) ([wb]) ([KQkq-]+) ([a-h1-8-]+) (\d+) (\d+)$")
    if my_color == 'white':
        piece2encoding = {
            'P' : 1,
            'R' : 2,
            'N' : 3,
            'B' : 4,
            'Q' : 5,
            'K' : 6,
        }
    elif my_color == 'black':
        piece2encoding = {
            'p' : 1,
            'r' : 2,
            'n' : 3,
            'b' : 4,
            'q' : 5,
            'k' : 6,
        }

    
    match = fen_pattern.match(fen_string)

    
    print(f'Input string: {fen_string}')
    
    if match:
        piece_placement, active_color, castling, en_passant, halfmove, fullmove = match.groups()
        print("Piece Placement:", piece_placement)
        print("Active Color:", active_color)
        print("Castling Rights:", castling)
        print("En Passant Target:", en_passant)
        print("Halfmove Clock:", halfmove)
        print("Fullmove Number:", fullmove)

        pieces_by_row = piece_placement.split('/')

        pieces_by_row = replace_numbers_with_u(pieces_by_row)

        # 7 is the value for unknown spaces.
        input_tensor = torch.full((8, 8), 7)

        for (rank, row) in zip(reversed(range(input_tensor.shape[0])), pieces_by_row):  # rows
            for (file, piece) in zip(reversed(range(input_tensor.shape[1])), row):  # columns
                if piece in piece2encoding.keys():
                    input_tensor[rank][file] = piece2encoding[piece]

        print(f'Input Tensor: {input_tensor}')

        return input_tensor
        
    else:
        print("Invalid FEN string")


input_tensor = FEN2InputTensor(game_history.truth_fen_before_move(arbitrary_move))

Input string: rnbqkbnr/pppp1ppp/8/4p3/8/8/PPPPPPPP/RNBQKBNR w KQkq e6 0 2
Piece Placement: rnbqkbnr/pppp1ppp/8/4p3/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: e6
Halfmove Clock: 0
Fullmove Number: 2
Input Tensor: tensor([[2, 3, 4, 6, 5, 4, 3, 2],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7]])


In [80]:
def UpdateInputWithSense(input_tensor, sensing_result):
    my_color = 'white'
    fen_pattern = re.compile(r"^([rnbqkpRNBQKP1-8/]+) ([wb]) ([KQkq-]+) ([a-h1-8-]+) (\d+) (\d+)$")
    if my_color == 'white':
        piece2encoding = {
            'p' : -1,
            'r' : -2,
            'n' : -3,
            'b' : -4,
            'q' : -5,
            'k' : -6,
            'e' : 0,
            'P' : 1,
            'R' : 2,
            'N' : 3,
            'B' : 4,
            'Q' : 5,
            'K' : 6,
        }
    elif my_color == 'black':
        piece2encoding = {
            'P' : -1,
            'R' : -2,
            'N' : -3,
            'B' : -4,
            'Q' : -5,
            'K' : -6,
            'e' : 0,
            'p' : 1,
            'r' : 2,
            'n' : 3,
            'b' : 4,
            'q' : 5,
            'k' : 6,
        }
    for (idx, piece) in sensing_result:
        rank, file = divmod(idx, 8)

        if piece != None:
            input_tensor[rank][file] = piece2encoding[piece.symbol()]
        else:
            input_tensor[rank][file] = piece2encoding['e']

    return input_tensor

sensing_result = game_history.sense_result(arbitrary_move)
print(sensing_result)

print(UpdateInputWithSense(input_tensor, sensing_result))

[(24, None), (25, None), (26, None), (16, None), (17, None), (18, None), (8, Piece.from_symbol('P')), (9, Piece.from_symbol('P')), (10, Piece.from_symbol('P'))]
tensor([[2, 3, 4, 6, 5, 4, 3, 2],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 7, 7, 7, 7, 7],
        [0, 0, 0, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7]])


In [89]:
def replace_numbers_with_e(strings):
    def replacer(match):
        return 'e' * int(match.group())  # Convert number to integer and repeat 'e' that many times

    return [re.sub(r'\d+', replacer, s) for s in strings]

# Write function to convert FEN string to Output Tensor
def FEN2OutputTensor(fen_string):
    my_color = 'white'
    fen_pattern = re.compile(r"^([rnbqkpRNBQKP1-8/]+) ([wb]) ([KQkq-]+) ([a-h1-8-]+) (\d+) (\d+)$")
    if my_color == 'white':
        opp_piece2channel_idx = {
            'e' : 0,   # Empty space
            'p' : 1,   # My pawn
            'r' : 2,   # My rook
            'n' : 3,   # My knight
            'b' : 4,   # My bishop
            'q' : 5,   # My queen
            'k' : 6,   # My king
            'P' : 7,   # Opp pawn
            'R' : 8,   # Opp rook
            'N' : 9,   # Opp knight
            'B' : 10,  # Opp bishop
            'Q' : 11,  # Opp queen
            'K' : 12   # Opp king
        }
    elif my_color == 'black':
        opp_piece2channel_idx = {
            'e' : 0,
            'P' : 1,
            'R' : 2,
            'N' : 3,
            'B' : 4,
            'Q' : 5,
            'K' : 6,
            'p' : -1,
            'r' : -2,
            'n' : -3,
            'b' : -4,
            'q' : -5,
            'k' : -6
        }

    
    match = fen_pattern.match(fen_string)
    
    if match:
        piece_placement, active_color, castling, en_passant, halfmove, fullmove = match.groups()
        print("Piece Placement:", piece_placement)
        print("Active Color:", active_color)
        print("Castling Rights:", castling)
        print("En Passant Target:", en_passant)
        print("Halfmove Clock:", halfmove)
        print("Fullmove Number:", fullmove)

        pieces_by_row = piece_placement.split('/')

        pieces_by_row = replace_numbers_with_e(pieces_by_row)
        print(f'modified fen string: {pieces_by_row}')

        output_tensor = torch.zeros((8, 8, 6 + 6 + 1))

        for (rank, fen_row) in zip(reversed(range(input_tensor.shape[0])), pieces_by_row):
            for (file, piece) in zip(reversed(range(input_tensor.shape[1])), fen_row):
                output_tensor[rank][file][opp_piece2channel_idx[piece]] = 1
        return output_tensor
        
    else:
        print("Invalid FEN string")


out = FEN2OutputTensor(game_history.truth_fen_before_move(arbitrary_move))
print(out[:, :, 12])

Piece Placement: rnbqkbnr/pppp1ppp/8/4p3/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: e6
Halfmove Clock: 0
Fullmove Number: 2
modified fen string: ['rnbqkbnr', 'ppppeppp', 'eeeeeeee', 'eeeepeee', 'eeeeeeee', 'eeeeeeee', 'PPPPPPPP', 'RNBQKBNR']
tensor([[0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
